In [1]:
import pandas as pd
import sys
import os
parent_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(parent_dir)

from utils import const

data_path = "../data/binance/BTCUSDT-1m-2024-all-with-indicators.csv"

df = pd.read_csv(data_path)
df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
df = df.set_index("open_time")

In [2]:
df["target"] = (df["close"].shift(-1) > df["close"]).astype(int)
df = df.dropna()

X = df[const.factor_cols]
y = df["target"]

X_train = X[X.index < const.split_time]
X_test = X[X.index >= const.split_time]
y_train = y[y.index < const.split_time]
y_test = y[y.index >= const.split_time]

In [3]:
import torch

import torch.nn as nn
import torch.optim as optim


class SimpleMLP(nn.Module):
    def __init__(self, input_dim):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x


X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)

model = SimpleMLP(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

input_weights = model.fc1.weight.detach().cpu().numpy()
factor_importance = abs(input_weights).mean(axis=0)
factor_ranking = sorted(
    zip(const.factor_cols, factor_importance), key=lambda x: x[1], reverse=True
)

for name, score in factor_ranking:
    print(f"{name}: {score:.4f}")

low: 0.0947
close: 0.0924
open: 0.0922
bb_bbm: 0.0921
bb_bbl: 0.0911
count: 0.0876
rsi: 0.0876
adx: 0.0875
high: 0.0873
taker_buy_volume: 0.0860
ema_20: 0.0842
donchian_high: 0.0837
quote_volume: 0.0831
mfi: 0.0824
obv: 0.0822
bb_bbh: 0.0818
vwap: 0.0796
atr: 0.0791
stoch_d: 0.0787
ignore: 0.0787
cci: 0.0778
williams_r: 0.0774
taker_buy_quote_volume: 0.0768
macd_signal: 0.0753
sma_20: 0.0748
macd: 0.0747
donchian_low: 0.0728
roc: 0.0722
stoch_k: 0.0702
macd_diff: 0.0700
volume: 0.0684
